In [ ]:
# pip install hypopt 

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from hypopt import GridSearch
from sklearn.linear_model import ElasticNet
import copy

In [ ]:
train = pd.read_csv('preprocessed_train.csv')
test = pd.read_csv('preprocessed_test.csv')

# Removing the last event from each case to reduce noise in the models

In [ ]:
train.drop(train.groupby('case concept:name').tail(1).index, axis=0, inplace=True)
test.drop(test.groupby('case concept:name').tail(1).index, axis=0, inplace=True)

# Evaluation


In [ ]:
def time_evaluation(y_test, y_pred, model: str):
    print('\n')
    print(f"Error metrics (measured in hours) for the {model} when predicting the next event's Unix timestamp")
    
    print('Mean Absolute Error:', round(mean_absolute_error(y_test, y_pred)/3600,2))
    print('Root Mean Squared Error:', round(np.sqrt(mean_squared_error(y_test, y_pred)/3600),2))
#     print('R2 score:', round(r2_score(y_test, y_pred),2))
    
    
def event_evaluation(y_test, y_pred, model: str, avg="weighted"):

    precision = precision_score(y_test, y_pred, average=avg, zero_division=0)
    recall = recall_score(y_test, y_pred, average=avg, zero_division=0)
    F1_score = f1_score(y_test, y_pred, average=avg, zero_division=0)
    accuracy = accuracy_score(y_test, y_pred)
    print('\n')
    print(f'Error metrics for the {model} when predicting the next event')
    
    print(f'Accuracy: {round(accuracy,2)}')
    print(f'Precision: {round(precision,2)}')
    print(f'Recall: {round(recall,2)}')
    print(f'f1-score: {round(F1_score,2)}')
#     print(confusion_matrix(y_test, y_pred))

# Data splitting and encoding

In [ ]:
train_objs_num = train.shape[0]

dataset = pd.concat(objs=[train, test], axis=0)

dataset = dataset.replace({'event lifecycle:transition': {'SCHEDULE': 0, 'START': 1, 'COMPLETE': 2}})
dataset = pd.get_dummies(dataset, prefix = ['current', 'prev', '2prev'], 
                           columns = ['event concept:name', 'prev_event', '2prev_event'])

train_OHE = copy.copy(dataset[:train_objs_num])
test_OHE = copy.copy(dataset[train_objs_num:])



val_OHE = train_OHE.loc[147054:]
train_OHE = train_OHE.loc[:147054]

first_val_event_timestamp = val_OHE['event time:timestamp'].min()
train_OHE = train_OHE.groupby('case concept:name').filter(lambda g: g['event time:timestamp'].min() < first_val_event_timestamp and g['event time:timestamp'].max() < first_val_event_timestamp)

first_test_event_timestamp = test_OHE['event time:timestamp'].min()
val_OHE = val_OHE.groupby('case concept:name').filter(lambda g: g['event time:timestamp'].min() < first_test_event_timestamp and g['event time:timestamp'].max() < first_test_event_timestamp)


In [ ]:
train_OHE.shape, val_OHE.shape, test_OHE.shape

In [ ]:
lst = []
for element in list(train_OHE.columns):
    if element not in list(test_OHE.columns):
        lst.append(element)
print('event type that happen in the train but not the test set: ' + str(lst))

In [ ]:
features_time = ['timestamp_finish', 'seconds_prev', 'active cases', 'day_week', 'time_of_day', 
                 'case AMOUNT_REQ', 'event lifecycle:transition', 
                 'current_A_ACCEPTED', 'current_A_ACTIVATED', 'current_A_APPROVED', 'current_A_CANCELLED', 'current_A_DECLINED', 'current_A_FINALIZED', 'current_A_PARTLYSUBMITTED', 'current_A_PREACCEPTED', 'current_A_REGISTERED', 'current_A_SUBMITTED', 'current_O_ACCEPTED','current_O_CANCELLED', 'current_O_CREATED', 'current_O_DECLINED','current_O_SELECTED', 'current_O_SENT', 'current_O_SENT_BACK','current_W_Afhandelen leads', 'current_W_Beoordelen fraude','current_W_Completeren aanvraag','current_W_Nabellen incomplete dossiers', 'current_W_Nabellen offertes','current_W_Valideren aanvraag', 'current_W_Wijzigen contractgegevens']

# features_time_test = ['timestamp_finish', 'seconds_prev', 'active cases', 'day_week', 'time_of_day', 
#                  'case AMOUNT_REQ', 'event lifecycle:transition', 
#            'current_A_ACCEPTED', 'current_A_ACTIVATED', 'current_A_APPROVED','current_A_CANCELLED', 'current_A_DECLINED', 'current_A_FINALIZED','current_A_PARTLYSUBMITTED', 'current_A_PREACCEPTED','current_A_REGISTERED', 'current_A_SUBMITTED', 'current_O_ACCEPTED', 'current_O_CANCELLED', 'current_O_CREATED', 'current_O_DECLINED','current_O_SELECTED', 'current_O_SENT', 'current_O_SENT_BACK','current_W_Afhandelen leads', 'current_W_Beoordelen fraude','current_W_Completeren aanvraag','current_W_Nabellen incomplete dossiers', 'current_W_Nabellen offertes', 'current_W_Valideren aanvraag']
features_time_test = features_time.copy()

target_time = 'seconds_next'

features_event = ['active cases', 'day_week', 'time_of_day',  
                  'event lifecycle:transition', 'case AMOUNT_REQ', 
       'current_A_ACCEPTED', 'current_A_ACTIVATED', 'current_A_APPROVED','current_A_CANCELLED', 'current_A_DECLINED', 'current_A_FINALIZED','current_A_PARTLYSUBMITTED', 'current_A_PREACCEPTED','current_A_REGISTERED', 'current_A_SUBMITTED', 'current_O_ACCEPTED','current_O_CANCELLED', 'current_O_CREATED', 'current_O_DECLINED','current_O_SELECTED', 'current_O_SENT', 'current_O_SENT_BACK','current_W_Afhandelen leads', 'current_W_Beoordelen fraude','current_W_Completeren aanvraag','current_W_Nabellen incomplete dossiers', 'current_W_Nabellen offertes','current_W_Valideren aanvraag', 'current_W_Wijzigen contractgegevens','prev_A_ACCEPTED', 'prev_A_ACTIVATED', 'prev_A_APPROVED','prev_A_CANCELLED', 'prev_A_DECLINED', 'prev_A_FINALIZED','prev_A_PARTLYSUBMITTED', 'prev_A_PREACCEPTED', 'prev_A_REGISTERED','prev_A_SUBMITTED', 'prev_FIRST EVENT', 'prev_O_ACCEPTED', 'prev_O_CANCELLED', 'prev_O_CREATED', 'prev_O_DECLINED','prev_O_SELECTED', 'prev_O_SENT', 'prev_O_SENT_BACK','prev_W_Afhandelen leads', 'prev_W_Beoordelen fraude','prev_W_Completeren aanvraag', 'prev_W_Nabellen incomplete dossiers','prev_W_Nabellen offertes', 'prev_W_Valideren aanvraag','prev_W_Wijzigen contractgegevens', '2prev_A_ACCEPTED','2prev_A_ACTIVATED', '2prev_A_APPROVED', '2prev_A_CANCELLED','2prev_A_FINALIZED', '2prev_A_PARTLYSUBMITTED', '2prev_A_PREACCEPTED','2prev_A_REGISTERED', '2prev_A_SUBMITTED', '2prev_FIRST EVENT','2prev_O_ACCEPTED', '2prev_O_CANCELLED', '2prev_O_CREATED','2prev_O_SELECTED', '2prev_O_SENT', '2prev_O_SENT_BACK','2prev_W_Afhandelen leads', '2prev_W_Beoordelen fraude','2prev_W_Completeren aanvraag', '2prev_W_Nabellen incomplete dossiers','2prev_W_Nabellen offertes', '2prev_W_Valideren aanvraag','2prev_W_Wijzigen contractgegevens']

# features_event_test = ['active cases', 'day_week', 'time_of_day',  
#                   'event lifecycle:transition', 'case AMOUNT_REQ', 
#                        'current_A_ACCEPTED', 'current_A_ACTIVATED', 'current_A_APPROVED','current_A_CANCELLED', 'current_A_DECLINED', 'current_A_FINALIZED','current_A_PARTLYSUBMITTED', 'current_A_PREACCEPTED','current_A_REGISTERED', 'current_A_SUBMITTED', 'current_O_ACCEPTED','current_O_CANCELLED', 'current_O_CREATED', 'current_O_DECLINED','current_O_SELECTED', 'current_O_SENT', 'current_O_SENT_BACK','current_W_Afhandelen leads', 'current_W_Beoordelen fraude','current_W_Completeren aanvraag','current_W_Nabellen incomplete dossiers', 'current_W_Nabellen offertes','current_W_Valideren aanvraag','prev_A_ACCEPTED', 'prev_A_ACTIVATED', 'prev_A_APPROVED','prev_A_CANCELLED', 'prev_A_DECLINED', 'prev_A_FINALIZED','prev_A_PARTLYSUBMITTED', 'prev_A_PREACCEPTED', 'prev_A_REGISTERED','prev_A_SUBMITTED', 'prev_FIRST EVENT', 'prev_O_ACCEPTED','prev_O_CANCELLED', 'prev_O_CREATED', 'prev_O_DECLINED','prev_O_SELECTED', 'prev_O_SENT', 'prev_O_SENT_BACK','prev_W_Afhandelen leads', 'prev_W_Beoordelen fraude','prev_W_Completeren aanvraag', 'prev_W_Nabellen incomplete dossiers','prev_W_Nabellen offertes', 'prev_W_Valideren aanvraag','2prev_A_ACCEPTED', '2prev_A_ACTIVATED', '2prev_A_APPROVED','2prev_A_FINALIZED', '2prev_A_PARTLYSUBMITTED', '2prev_A_PREACCEPTED','2prev_A_REGISTERED', '2prev_A_SUBMITTED', '2prev_FIRST EVENT','2prev_O_ACCEPTED', '2prev_O_CANCELLED', '2prev_O_CREATED','2prev_O_SELECTED', '2prev_O_SENT', '2prev_O_SENT_BACK','2prev_W_Afhandelen leads', '2prev_W_Beoordelen fraude','2prev_W_Completeren aanvraag', '2prev_W_Nabellen incomplete dossiers','2prev_W_Nabellen offertes', '2prev_W_Valideren aanvraag']
features_event_test = features_event.copy()

target_event = 'next_event'


# TIME
X_train_time = train_OHE[features_time]
y_train_time = train_OHE[target_time]

X_val_time = val_OHE[features_time]
y_val_time = val_OHE[target_time]

X_test_time = test_OHE[features_time_test].copy()
y_test_time = test_OHE[target_time]


# EVENT
X_train_event = train_OHE[features_event]
y_train_event = train_OHE[target_event]

X_val_event = val_OHE[features_event]
y_val_event = val_OHE[target_event]

X_test_event = test_OHE[features_event_test].copy()
y_test_event = test_OHE[target_event]



# X_test_time['current_W_Wijzigen contractgegevens']=-1
# X_test_event[['current_W_Wijzigen contractgegevens', 'prev_W_Wijzigen contractgegevens', '2prev_A_CANCELLED', '2prev_W_Wijzigen contractgegevens']]=-1

In [ ]:
X_train_event.shape, X_train_time.shape, X_test_event.shape, X_test_time.shape

In [ ]:
# ohe = OneHotEncoder().fit(train['event concept:name'].to_numpy().reshape(-1, 1))

# transformed = ohe.transform(train['event concept:name'].to_numpy().reshape(-1, 1))
# train_OHE = pd.DataFrame(transformed, columns=jobs_encoder.get_feature_names())

# Event prediction

In [ ]:
def RandomForestEvents(X_train, y_train, X_val, y_val):
    
    
    params={'max_depth': [None],
     'n_estimators': [500, 800, 1200, 1800]}

    
    forest_clf = RandomForestClassifier(bootstrap=True, criterion='gini', max_features='sqrt', random_state=42)    
    
    grid = GridSearch(model=forest_clf, param_grid=params, parallelize=False)

    grid.fit(X_train, y_train, X_val, y_val)
    
    return grid.best_estimator_


In [ ]:
rf_event = RandomForestEvents(X_train_event, y_train_event, X_val_event, y_val_event)

In [ ]:
rf_event_train = rf_event.predict(X_train_event)
rf_event_val = rf_event.predict(X_val_event)
rf_event_test = rf_event.predict(X_test_event)

In [ ]:
print(rf_event.get_params())

event_evaluation(y_train_event, rf_event_train, 'RF EVENT TRAIN')
event_evaluation(y_val_event, rf_event_val, 'RF EVENT VAL')
event_evaluation(y_test_event, rf_event_test, 'RF EVENT TEST')

In [ ]:
rf_event_features = dict(zip(features_event, rf_event.feature_importances_))

rf_event_features

# some categories of current, prev, 2prev event are very useful for next event -> current, prev, 2prev are good features

# Time prediction

In [ ]:
def RandomForestTime(X_train, y_train, X_val, y_val):
    
    
    params={'max_depth': [None],
     'n_estimators': [500, 800, 1200, 1800]}

    forest_reg = RandomForestRegressor(bootstrap = True, max_features='sqrt', random_state=42)
    
    grid = GridSearch(model=forest_reg, param_grid=params, parallelize=False)

    grid.fit(X_train, y_train, X_val, y_val)
    
    return grid.best_estimator_


In [ ]:
rf_time = RandomForestTime(X_train_time, y_train_time, X_val_time, y_val_time)

In [ ]:
rf_time_train = rf_time.predict(X_train_time)
rf_time_val = rf_time.predict(X_val_time)
rf_time_test = rf_time.predict(X_test_time)

In [ ]:
print(rf_time.get_params())

time_evaluation(y_train_time, rf_time_train, 'RF TIME TRAIN')
time_evaluation(y_val_time, rf_time_val, 'RF TIME VAL')
time_evaluation(y_test_time, rf_time_test, 'RF TIME TEST')

In [ ]:
rf_time_features = dict(zip(features_time, rf_time.feature_importances_))

rf_time_features

# current event, current and prev time, note that active cases is relatively unimportant here too, mention cyclical time too

# Elastic net

In [ ]:
def ElasticNetTime(X_train, y_train, X_val, y_val):
    
    
    params = {"l1_ratio": [.1, .3 ,.5,.85,.95, .99, 1],
              'alpha':[0.1,0.3,0.5,0.7,0.9,1],
             'max_iter':[5000]}

#     params = {"l1_ratio": [.5],
#               'alpha':[.5]}


    en_reg = ElasticNet(random_state=42)
    
    grid = GridSearch(model=en_reg, param_grid=params, parallelize=False)

    grid.fit(X_train, y_train, X_val, y_val)
    
    return grid.best_estimator_


In [ ]:
en_time = ElasticNetTime(X_train_time, y_train_time, X_val_time, y_val_time)

In [ ]:
en_time_train = en_time.predict(X_train_time)
en_time_val = en_time.predict(X_val_time)
en_time_test = en_time.predict(X_test_time)

In [ ]:
print(en_time.get_params())

time_evaluation(y_train_time, en_time_train, 'EN TIME TRAIN')
time_evaluation(y_val_time, en_time_val, 'EN TIME VAL')
time_evaluation(y_test_time, en_time_test, 'EN TIME TEST')

In [ ]:
en_time_features = dict(zip(features_time, en_time.coef_))

en_time_features

# Decision Tree event

In [ ]:
def DecisionTreeEvent(X_train, y_train, X_val, y_val):


    params={'max_depth': [8, 16, 32, 64, 128, None]}

    tree_clf = DecisionTreeClassifier(random_state=42)
    
    grid = GridSearch(model=tree_clf, param_grid=params, parallelize=False)

    grid.fit(X_train, y_train, X_val, y_val)
    
    return grid.best_estimator_



In [ ]:
dt_event = DecisionTreeEvent(X_train_event, y_train_event, X_val_event, y_val_event)

In [ ]:
dt_event_train = dt_event.predict(X_train_event)
dt_event_val = dt_event.predict(X_val_event)
dt_event_test = dt_event.predict(X_test_event)

In [ ]:
print(dt_event.get_params())

event_evaluation(y_train_event, dt_event_train, 'DT EVENT TRAIN')
event_evaluation(y_val_event, dt_event_val, 'DT EVENT VAL')
event_evaluation(y_test_event, dt_event_test, 'DT EVENT TEST')

In [ ]:
dt_event_features = dict(zip(features_event, dt_event.feature_importances_))

dt_event_features

In [ ]:
dt_event.get_depth()

# Decision tree time

In [ ]:
def DecisionTreeTime(X_train, y_train, X_val, y_val):

    params={'max_depth': [8, 16, 32, 64, 128, None]}

    tree_reg = DecisionTreeRegressor(random_state=42)
    
    grid = GridSearch(model=tree_reg, param_grid=params, parallelize=False)

    grid.fit(X_train, y_train, X_val, y_val)
    
    return grid.best_estimator_



In [ ]:
dt_time = DecisionTreeTime(X_train_time, y_train_time, X_val_time, y_val_time)

In [ ]:
dt_time_train = dt_time.predict(X_train_time)
dt_time_val = dt_time.predict(X_val_time)
dt_time_test = dt_time.predict(X_test_time)

In [ ]:
print(dt_time.get_params())

time_evaluation(y_train_time, dt_time_train, 'DT TIME TRAIN')
time_evaluation(y_val_time, dt_time_val, 'DT TIME VAL')
time_evaluation(y_test_time, dt_time_test, 'DT TIME TEST')

In [ ]:
dt_time_features = dict(zip(features_time, dt_time.feature_importances_))

dt_time_features

# active cases is stronger here

In [ ]:
dt_time.get_depth()

## Time, Memory and CPU usage

### Memory and CPU usage

In [ ]:
import psutil
import time

# start measuring CPU and memory usage
process = psutil.Process()

# start point
start_time = time.time()

forest_event, y_pred_event = RandomForestEvents(X_train, X_test, y_train, y_test)
event_evaluation(y_test['next_event'], y_pred_event, 'Random forest classifier')
# forest_time, y_pred_time = RandomForestTime(X_train, X_test, y_train, y_test)
# time_evaluation(y_test['next_timestamp'], y_pred_time, 'Random forest regressor')
# end point
end_time = time.time()

# calculate time taken
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")

# measure CPU and memory usage
cpu_usage = process.cpu_percent()
memory_usage = process.memory_info().rss / 1024 / 1024  # in MB
print(f"CPU usage: {cpu_usage:.2f}%")
print(f"Memory usage: {memory_usage:.2f} MB")

### Time usage

In [ ]:
start_time = time.time()

# RandomForestEvents function
forest_event, y_pred_event = RandomForestEvents(X_train, X_test, y_train, y_test)

# event_evaluation function
event_evaluation(y_test['next_event'], y_pred_event, 'Random forest classifier')

print(f"Time taken by RandomForestEvents: {time.time() - start_time} seconds")

start_time = time.time()

# RandomForestTime function
forest_time, y_pred_time = RandomForestTime(X_train, X_test, y_train, y_test)

# # time_evaluation function
# time_evaluation(y_test['next_timestamp'], y_pred_time, 'Random forest regressor')

print(f"Time taken by RandomForestTime: {time.time() - start_time} seconds")

# Visualizations

### Confusion matrix for Random Forest estimator

In [ ]:
cm = confusion_matrix(y_test['next_event'], y_pred_event, normalize='true')

# plot confusion matrix as a heatmap
sns.set(font_scale=1.2)
fig, ax = plt.subplots(figsize=(10,8), facecolor='w')  # add facecolor attribute
sns.heatmap(cm, fmt='g', cmap='RdBu_r', ax=ax, annot_kws={"size": 14})
ax.set_xlabel('Predicted labels', fontsize=14)
ax.set_ylabel('True labels', fontsize=14)
ax.set_title('Confusion matrix for Random Forest estimator', fontsize=16)
plt.show()

### Naive and Random Forest estimators accuracy and precision

In [ ]:
# plot the naive and random forest f1, acc, pre, recall
metrics_df = pd.DataFrame({
    ' ': ['F1 Score', 'Accuracy', 'Precision', 'Recall'],
    'Naive': ['0.266', '0.346', '0.297', '0.346'],
    'Random Forest': ['0.748', '0.785', '0.772', '0.785']    
})

metrics_df.to_string(index=False)
metrics_df

### Bar chart for feature importance of Random Forest event estimator

In [ ]:
import matplotlib.pyplot as plt

# get feature importances and their names
importances = forest_event.feature_importances_
# feature_names = X_train.columns.values
feature_names = ['Current event', 'Previous event', '2nd previous event']

# sort the features by importance in descending order
indices = np.argsort(importances)[::-1]
sorted_importances = importances[indices]

plt.figure(figsize=(10,6))
plt.bar(feature_names, sorted_importances, color=['brown', 'green', 'blue'])
plt.title('Feature Importances of Random Forest event estimator')
plt.xlabel('Features')
plt.xticks(rotation=0)
plt.ylabel('Importance')
plt.show()


### Bar chart feature importance Random Forest time estimator

In [ ]:
# get feature importances and their names
importances = forest_time.feature_importances_
feature_names = ['Current event time','Prev event time', 'Prev event', '2nd prev event', 'Current event']

# sort the features by importance in descending order
indices = np.argsort(importances)[::-1]
sorted_importances = importances[indices]

plt.figure(figsize=(10,6))
plt.bar(feature_names, sorted_importances, color=['brown', 'green', 'blue', 'yellow'])
plt.title('Feature Importances of Random Forest time estimator')
plt.xlabel('Features')
plt.xticks(rotation=0)
plt.ylabel('Importance')
plt.show()

In [ ]:
residual = pd.concat([y_test, pd.Series(y_pred_time)], axis="columns", ignore_index=True)
residual.columns = ['none','real','pred']
residual['error']= residual['real']-residual['pred']

sns.residplot(x='real', y='error', data=residual)
